In [1]:
from itertools import combinations, product
import os
import pickle
import warnings

import joblib
from joblib import Memory
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import KernelPCA, TruncatedSVD
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, \
    GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from stability_selection import StabilitySelection

In [2]:
seed = 15
np.random.seed(seed)
n_jobs = joblib.cpu_count() - 1

In [3]:
with open("dataset.pkl", "rb") as f:
    DATA = pickle.load(f)
    pheno = DATA["pheno"]
    X_gpa = DATA["X_gpa"]
    X_snps = DATA["X_snps"]
    X_genexp = StandardScaler().fit_transform(DATA["X_genexp"])
X = np.concatenate([X_gpa, X_snps, X_genexp], axis=1)

gpa_idx = np.arange(0, X_gpa.shape[1] - 1)
snps_idx = np.arange(0, X_snps.shape[1] - 1) + gpa_idx[-1] + 1
genexp_idx = np.arange(0, X_genexp.shape[1] - 1) + snps_idx[-1] + 1

Construit un **pipeline** permettant :

 1. d'appliquer d'éventuelles transformations aux matrices de régresseurs,
 2. de procéder à une potentielle réduction de dimension, par matrice de régresseur (y compris *drop* pour ne pas sélectionner la matrice),
 3. d'effectuer une réduction de dimension sur les matrices concaténées,
 4. et enfin d'entrainer un classifieur.

Il est ensuite possible de faire varier les combinaisons de transformations, méthodes de réduction de dimension, classifieurs ; ainsi que leurs hyper-paramètres.

In [4]:
trans_ind = ColumnTransformer(transformers=[("genexp", StandardScaler(), genexp_idx)],
                              remainder="passthrough")
dim_red_ind = ColumnTransformer(transformers=[("gpa", "passthrough", gpa_idx), # drop, passthrough, KernelPCA, StabSel, TruncatedSVD
                                              ("snps", "passthrough", snps_idx), # drop, passthrough, KernelPCA, StabSel, TruncatedSVD
                                              ("genexp", "passthrough", genexp_idx)]) # drop, passthrough, KernelPCA, StabSel, TruncatedSVD

# Fitting the dimensionality reduction transformers might be expansive, so we use caching
cachepath = ".cache"
if not os.path.exists(cachepath):
    os.mkdir(cachepath)

pipe = Pipeline([("trans_ind", trans_ind), ("dim_red_ind", dim_red_ind),
                 ("dim_red", "passthrough"), # passthrough, KernelPCA, StabSel, TruncatedSVD
                 ("clf", DummyClassifier())],       # LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis, AdaBoostClassifier,
                memory=Memory(location=cachepath))  # BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier,
                                                    # LogisticRegression, PassiveAggressiveClassifier, Perceptron, SGDClassifier, KNeighborsClassifier,
                                                    # LinearSVC, SVC

In [5]:
pipe

Pipeline(memory=Memory(location=.cache/joblib),
         steps=[('trans_ind',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('genexp', StandardScaler(),
                                                  array([88239, 88240, 88241, ..., 94261, 94262, 94263]))])),
                ('dim_red_ind',
                 ColumnTransformer(transformers=[('gpa', 'passthrough',
                                                  array([    0,     1,     2, ..., 16001, 16002, 16003])),
                                                 ('snps', 'passthrough',
                                                  array([16004, 16005, 16006, ..., 88236, 88237, 88238])),
                                                 ('genexp', 'passthrough',
                                                  array([88239, 88240, 88241, ..., 94261, 94262, 94263]))])),
                ('dim_red', 'passthrough'), ('clf', DummyClassifier())])

Pour limiter le nombres d'hyper-paramètres à ajuster par validation croisée, dans un premier temps, on cherche à sélectionner la bonne procédure de transformation et de réduction de dimension sans faire varier les hyper-paramètres des classifieurs. Dans un second temps on fait varier les hyper-paramètres des classifieurs en gardant la procédure de pré-traitement la plus performante de l'étape précédente.

In [6]:
# function that create the grids corresponding to the possible roots and parameters
def create_grid(roots, params):
    # recursively adds parameters to a grid
    def add_to_grid(g, r, p):
        if len(p[0]) > 0:
            r = "__".join([r, p[0]])
        g[r] = p[1]
        # parameters can have inner parameters
        for c in p[2]:
            add_to_grid(g, r, c)

    grids = []
    # loops through all combinations of root / param pairs of size len(roots)
    for comb in combinations(product(roots, params), len(roots)):
        valid = True
        grid = {}
        for root, param in comb:
            # a combination is valid only if a root appears only once
            if root in grid:
                valid = False
                break
            else:
                add_to_grid(grid, root, param)
        if valid:
            grids.append(grid)
    return grids

In [7]:
dim_red_ind_grid_roots = ["dim_red_ind__gpa", "dim_red_ind__snps", "dim_red_ind__genexp"]
dim_red_ind_grid_params = [("", ["drop", "passthrough"], []),
                           ("", [TruncatedSVD(random_state=seed)],
                            [("n_components", [64, 128, 256], [])]),
                           ("", [KernelPCA(random_state=seed)],
                            [("kernel", ["linear", "poly", "rbf", "sigmoid"], []),
                             ("n_components", [64, 128, 256], [])]),
                           ("", [StabilitySelection(random_state=seed)],
                            [("threshold", np.linspace(.6, .9, 4), [])])]
dim_red_ind_grid = create_grid(dim_red_ind_grid_roots, dim_red_ind_grid_params)
len(ParameterGrid(dim_red_ind_grid))

9261

In [8]:
dim_red_grid_roots = ["dim_red"]
dim_red_grid_params = [("", ["passthrough",], []),
                       ("", [TruncatedSVD(random_state=seed),],
                        [("n_components", [64, 128, 256], [])]),
                       ("", [KernelPCA(random_state=seed),],
                        [("kernel", ["linear", "poly", "rbf", "sigmoid"], []),
                         ("n_components", [64, 128, 256], [])]),
                       ("", [StabilitySelection(random_state=seed),],
                        [("threshold", np.linspace(.6, .9, 4), [])])]
dim_red_grid = create_grid(dim_red_grid_roots, dim_red_grid_params)
len(ParameterGrid(dim_red_grid))

20

In [9]:
clf_grid_roots = ["clf"]
clf_grid_params = [("", [AdaBoostClassifier(random_state=seed), GradientBoostingClassifier(random_state=seed)],
                    [("learning_rate", np.logspace(-2, 0, 3), [])]),
                   ("", [BaggingClassifier(random_state=seed)],
                    [("max_features", np.linspace(1/3, 2/3, 3), [])]),
                   ("", [ExtraTreesClassifier(bootstrap=True, oob_score=True, class_weight="balanced", random_state=seed),
                         RandomForestClassifier(oob_score=True, class_weight="balanced", random_state=seed)],
                    [("criterion", ["gini", "log_loss"], [])]),
                   ("", [LogisticRegression(class_weight="balanced", max_iter=1000, random_state=seed),
                         PassiveAggressiveClassifier(class_weight="balanced", random_state=seed)],
                    [("C", np.logspace(-1, 1, 3), [])]),
                   ("", [Perceptron(class_weight="balanced", random_state=seed), SGDClassifier(class_weight="balanced", random_state=seed)],
                    [("alpha", np.logspace(-5, -3, 3), [])]),
                   ("", [SVC(class_weight="balanced", max_iter=10000, random_state=seed)],
                    [("C", np.logspace(-1, 1, 3), []), ("kernel", ["linear", "poly", "rbf", "sigmoid"], [])])]
clf_grid = create_grid(clf_grid_roots, clf_grid_params)
len(ParameterGrid(clf_grid))

37

In [10]:
def merge_grids(grids):
    merged_grid = grids.pop()
    for grid in grids:
        merged_grid = [{**g1, **g2} for g1, g2 in product(merged_grid, grid)]
    return merged_grid

In [11]:
full_grid = merge_grids([dim_red_ind_grid, dim_red_grid, clf_grid])
len(ParameterGrid(full_grid))

6853140

In [12]:
cv_grid = GridSearchCV(pipe, full_grid, scoring="balanced_accuracy", n_jobs=n_jobs, pre_dispatch="n_jobs", verbose=2)

# Colistin

In [13]:
y = pheno["Colistin"].to_numpy()

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    cv_grid_colistin = cv_grid.fit(X, y)